In [1]:
import jitr

import rxmc

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [2]:
jitr.__version__

'2.5'

In [3]:
import lzma

import corner
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from tqdm import tqdm

## Define reaction participants

In [4]:
from exfor_tools import curate

In [5]:
Ca40 = (40, 20)
Ca48 = (48, 20)
targets = [Ca40, Ca48]
p = (1, 1)
n = (1, 0)
projectiles = [n, p]
reactions = [
    jitr.reactions.ElasticReaction(projectile=proj, target=targ)
    for proj in projectiles
    for targ in targets
]
reactions

[40-Ca(n,el), 48-Ca(n,el), 40-Ca(p,el), 48-Ca(p,el)]

In [6]:
quantities = {n: ["dXS/dA", "Ay"], p: ["dXS/dA", "dXS/dRuth", "Ay"]}

In [8]:
for reaction in reactions:
    curate.MultiQuantityReactionData(
        reaction=reaction,
        quantities=quantities[tuple(reaction.projectile)],
        settings={
            "Einc_range": [10,400],
            "filter_kwargs": {"min_num_pts": 5, "allow_cos" : False},
        },
        vocal=True,
    )


Now parsing dXS/dA for 40-Ca(n,el)

Found subentry 12785002 with the following columns:
['EN', 'EN-RSL', 'ANG-CM', 'DATA-CM', 'DATA-ERR']
Found subentry 12996002 with the following columns:
['EN', 'EN-RSL', 'ANG-CM', 'DATA', 'DATA-ERR']
Found subentry 13946003 with the following columns:
['EN', 'EN-ERR', 'ANG-CM', 'DATA-CM', 'ERR-S', 'ERR-SYS']
Found subentry 14303002 with the following columns:
['EN', 'EN-ERR', 'ANG-CM', 'DATA-CM', 'DATA-ERR']
Found subentry 14303003 with the following columns:
['EN', 'EN-ERR', 'ANG-CM', 'DATA-CM', 'DATA-ERR']

Succesfully parsed 4 entries
Failed to parse 0 entries:


Now parsing Ay for 40-Ca(n,el)

Found subentry 12785003 with the following columns:
['POL-BM', 'EN', 'EN-ERR', 'ANG-CM', 'DATA-CM', 'DATA-ERR']
Found subentry 12996004 with the following columns:
['POL-BM', 'EN', 'EN-RSL', 'ANG-CM', 'DATA', 'DATA-ERR']

Succesfully parsed 2 entries
Failed to parse 0 entries:


Now parsing dXS/dA for 48-Ca(n,el)

Found subentry 14303004 with the followin

## Optical potential model form

In [6]:
from jitr.optical_potentials.potential_forms import (
    thomas_safe,
    woods_saxon_prime_safe,
    woods_saxon_safe,
)


def central_potential(r, Vv, Wv, Rv, av, Wd, Rd, ad):
    return -(1j * Wv + Vv) * woods_saxon_safe(r, Rv, av) - (
        4j * ad * Wd
    ) * woods_saxon_prime_safe(r, Rd, ad)


def spin_orbit_potential(r, Vso, Wso, Rso, aso):
    return (
        (Vso + 1j * Wso)
        / jitr.utils.constants.WAVENUMBER_PION**2
        * thomas_safe(r, Rso, aso)
    )